<a href="https://colab.research.google.com/github/RosaGaLl/Data-Science_Machine-Learning/blob/main/Data_science_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MNIST digits classification dataset**


## **1. Descripción del problema de negocio**

El presente dataset recopila las opiniones de los usuarios sobre películas, las cuales pueden influir considerablemente en las decisiones de otros usuarios al momento de elegir una película nueva.Puesto que, si ahondamos más en estas reseñas podemos deducir que contienen valiosos sentimientos que reflejan la opinión de los espectadores.

Considerando ello, el contar con un sistema automático de análisis de sentimientos puede ayudar a plataformas de streaming, productoras o sitios de críticas a conocer la percepción general del público ante las películas vistas. Asimismo, el poder analizar estos textos y clasificarlos como positivos o negativos permite a estas mismas empresas entender mejor la recepción de los productos cinematográficos que se ofrecen, incluse se podrían ajustar estrategias de marketing o incluso ayudar a sus sistemas de recomendación.

## **2. Objetivo general**

El objetivo general de este proyecto es poder construir y mejorar
una red neuronal profunda que permita clasificar automáticamente las reseñas de películas como positivas o negativas, utilizando el dataset.
Y para ello, se usarán técnicas de Deep Learning aplicadas a Procesamiento de Lenguaje Natural (NLP), lo que permitirá que sea posible poder automatizar los análisis de los comentarios de las películas.

## **3.Origen de los datos**

El dataset a usar se encuentra disponible directamente en Keras a través del módulo keras.datasets.imdb.
Este contiene 25.000 reseñas de entrenamiento y 25.000 de prueba, etiquetadas como positivas = 1 o negativas =0. Las reseñas ya están preprocesadas y tokenizadas como secuencias de índices de palabras, lo cual nos permite obtener un análisis más detallado.

## **4. Definición de las variables**



*   x_train, x_test: Listas de secuencias de enteros, donde cada entero representa una palabra
*   y_train, y_test: Listas de etiquetas binarias las cuales son: 0 = negativo y 1 = positivo
*   num_words: Número de palabras más frecuentes que se conservarán en el vocabulario.
*   Longitud de las secuencias: Se da a conocer un maxlen para uniformar la longitud de entrada.
*  Las reseñas se encuentran tokenizadas



## **5.Librerías a usar en el proyecto**

Instalación de Librerías

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer




## **6.Desarrollo**

**a) Carga del dataset**

In [11]:
#Se carga el dataset con solo las 10000 palabras más frecuentes para tener un análisis más profundo y detallado
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)


# Es la longitud máxima de secuencia de las reseñas que se tienen
maxlen = 200
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


**b) Análisis exploratorio + insights**

In [3]:
print(f"Número de reseñas de entrenamiento: {len(x_train)}")
print(f"Número de reseñas de prueba: {len(x_test)}")
print(f"Longitud media de las reseñas: {np.mean([len(x) for x in x_train]):.2f}")
print(f"Distribución de clases en entrenamiento: {np.unique(y_train, return_counts=True)}")


Número de reseñas de entrenamiento: 25000
Número de reseñas de prueba: 25000
Longitud media de las reseñas: 200.00
Distribución de clases en entrenamiento: (array([0, 1]), array([12500, 12500]))


De los insights se tiene que:

Las clases están balanceadas: 50% positivas, 50% negativas.

Las reseñas tienen longitudes variadas, de ahí la necesidad de pad_sequences.

Solo las 10.000 palabras más frecuentes se usan para reducir complejidad.

**c) Red neuronal sencilla**

In [4]:
model = Sequential([
    Embedding(input_dim=num_words, output_dim=64, input_length=maxlen),
    LSTM(64),  # capa recurrente
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 50s 151ms/step - accuracy: 0.6869 - loss: 0.5729 - val_accuracy: 0.8714 - val_loss: 0.3167
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 78s 138ms/step - accuracy: 0.9002 - loss: 0.2555 - val_accuracy: 0.8702 - val_loss: 0.3144
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 46s 147ms/step - accuracy: 0.9318 - loss: 0.1849 - val_accuracy: 0.8710 - val_loss: 0.3289


**d) Evaluación**

In [5]:
results = model.evaluate(x_test, y_test)
print(f"Precisión en test: {results[1]*100:.2f}%")


782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.8647 - loss: 0.3410
Precisión en test: 86.66%


## **7. Profundización en Deep Learning (Mejora del modelo)**

In [14]:
# Desde aquí se define la arquitectura del modelo con capas adicionales, se usa la capa convolucional para captar patrones locales,capa densa y la capa de salida binaria
model_dl = Sequential([
    Embedding(input_dim=num_words, output_dim=64, input_length=maxlen),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


# Se compila el modelo
model_dl.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Se procede seguidamente a entrenar la red
history_dl = model_dl.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)



Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 62s 180ms/step - accuracy: 0.6531 - loss: 0.5864 - val_accuracy: 0.8658 - val_loss: 0.3518
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 58s 187ms/step - accuracy: 0.8986 - loss: 0.2771 - val_accuracy: 0.8652 - val_loss: 0.3263
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 80s 182ms/step - accuracy: 0.9337 - loss: 0.1911 - val_accuracy: 0.8780 - val_loss: 0.2987
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 59s 188ms/step - accuracy: 0.9607 - loss: 0.1256 - val_accuracy: 0.8750 - val_loss: 0.3665
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 193ms/step - accuracy: 0.9703 - loss: 0.0939 - val_accuracy: 0.8718 - val_loss: 0.3806


***Comparación de resultados***

In [10]:
loss_base, acc_base = model.evaluate(x_test, y_test)
loss_dl, acc_dl = model_dl.evaluate(x_test, y_test)

print(f"Modelo sencillo - Precisión: {acc_base*100:.2f}%")
print(f"Modelo mejorado - Precisión: {acc_dl*100:.2f}%")


782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.8647 - loss: 0.3410
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 24ms/step - accuracy: 0.8528 - loss: 0.4880
Modelo sencillo - Precisión: 86.66%
Modelo mejorado - Precisión: 85.54%


In [3]:
# Cargar solo las 10,000 palabras más frecuentes
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)



17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_test shape: (25000, 200)
y_test shape: (10000,)


Se da el preprocesamiento ( Rellenado de reseñas para que tengan la misma longitud)

In [5]:
maxlen = 200  # Cortamos o rellenamos a 200 palabras por reseña
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


Exploración del dataset

In [6]:
print(f"Número de reseñas de entrenamiento: {len(x_train)}")
print(f"Ejemplo de reseña (como secuencia de índices):\n{x_train[0]}")
print(f"Etiqueta (0=negativa, 1=positiva): {y_train[0]}")


Número de reseñas de entrenamiento: 25000
Ejemplo de reseña (como secuencia de índices):
[   5   25  100   43  838  112   50  670    2    9   35  480  284    5
  150    4  172  112  167    2  336  385   39    4  172 4536 1111   17
  546   38   13  447    4  192   50   16    6  147 2025   19   14   22
    4 1920 4613  469    4   22   71   87   12   16   43  530   38   76
   15   13 1247    4   22   17  515   17   12   16  626   18    2    5
   62  386   12    8  316    8  106    5    4 2223 5244   16  480   66
 3785   33    4  130   12   16   38  619    5   25  124   51   36  135
   48   25 1415   33    6   22   12  215   28   77   52    5   14  407
   16   82    2    8    4  107  117 5952   15  256    4    2    7 3766
    5  723   36   71   43  530  476   26  400  317   46    7    4    2
 1029   13  104   88    4  381   15  297   98   32 2071   56   26  141
    6  194 7486   18    4  226   22   21  134  476   26  480    5  144
   30 5535   18   51   36   28  224   92   25  104    4  22

Decodificar una reseña para ver el texto

In [9]:
# Cargar diccionario de palabras
word_index = imdb.get_word_index()
index_from = 3
word_index = {k:(v+index_from) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<OOV>"] = 2

# Invertir el diccionario
reverse_index = {v:k for k,v in word_index.items()}

# Decodificar
decoded_review = ' '.join([reverse_index.get(i, "?") for i in x_train[0]])
print(decoded_review)


and you could just imagine being there robert <OOV> is an amazing actor and now the same being director <OOV> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <OOV> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <OOV> to the two little boy's that played the <OOV> of norman and paul they were just brilliant children are often left out of the <OOV> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was

Armar una red neuronal sencilla (con embedding)

In [10]:
model = keras.Sequential([
    keras.layers.Embedding(input_dim=num_words, output_dim=32, input_length=maxlen),
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Salida binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Entrenar el modelo

In [11]:
history = model.fit(x_train, y_train, epochs=5, batch_size=512, validation_split=0.2)


Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.1026 - loss: -156.1726 - val_accuracy: 0.1018 - val_loss: -2414.8191
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.0998 - loss: -5093.9731 - val_accuracy: 0.1018 - val_loss: -18252.2148
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.1026 - loss: -27138.1074 - val_accuracy: 0.1018 - val_loss: -61530.9766
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.1013 - loss: -80740.0625 - val_accuracy: 0.1018 - val_loss: -145875.4531
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.1041 - loss: -182274.5312 - val_accuracy: 0.1018 - val_loss: -282935.6562


Evaluar y graficar resultados

In [12]:
results = model.evaluate(x_test, y_test)
print(f"Precisión en test: {results[1]*100:.2f}%")

# Graficar entrenamiento
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.legend()
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.title("Evolución de la precisión")
plt.show()


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 25000
'y' sizes: 10000


In [7]:
# Cargar diccionario de palabras
word_index = imdb.get_word_index()
index_from = 3
word_index = {k:(v+index_from) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<OOV>"] = 2

# Invertir el diccionario
reverse_index = {v:k for k,v in word_index.items()}

# Decodificar
decoded_review = ' '.join([reverse_index.get(i, "?") for i in x_train[0]])
print(decoded_review)


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
and you could just imagine being there robert <OOV> is an amazing actor and now the same being director <OOV> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <OOV> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <OOV> to the two little boy's that played the <OOV> of norman and paul they were just brilliant children are often left out of the <OOV> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it w

Decodificar una reseña para ver el texto

In [8]:
from tensorflow.keras.datasets import fashion_mnist

# Cargar datos
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
